In [1]:
import os
import sys
!export PYTHONPATH="${PYTHONPATH}:~/gt/3D_tracking/DEV/Tracking/nuscenes"
import numpy as np

from sklearn.utils.linear_assignment_ import linear_assignment

from nuscenes import NuScenes
from nuscenes.eval.common.config import config_factory
from nuscenes.eval.tracking.evaluate import TrackingEval
from nuscenes.eval.detection.data_classes import DetectionConfig
from nuscenes.eval.detection.data_classes import DetectionBox
from nuscenes.eval.tracking.data_classes import TrackingBox
from nuscenes.eval.common.loaders import load_prediction, load_gt, filter_eval_boxes
from nuscenes.eval.tracking.loaders import create_tracks
from pyquaternion import Quaternion
#from main import iou3d, convert_3dbox_to_8corner
import argparse
import pickle 
from utils.dict import *
from utils.stats import add_center_dist
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt

In [2]:
training_file_name = "/media/liangxu/ArmyData/nuscenes/Tracking_result/tracking_tmp/training_data_moving"
training_data = pickle.load(open(training_file_name , 'rb'))


## Generated Corrected Measurements

In [40]:
from tqdm import tqdm

In [3]:
print("Length of the training data ", len(training_data))

Length of the training data  934


In [11]:
print((training_data[0][0]).keys())

dict_keys(['sample_annotation', 'sample', 'sample_data_lidar', 'ego_pose', 'detection'])


In [16]:
print(training_data[0][0]['sample_annotation'].keys())
print(training_data[0][0]['sample_annotation']['translation'])
print(training_data[0][0]['sample_annotation']['rotation'])

dict_keys(['token', 'sample_token', 'instance_token', 'visibility_token', 'attribute_tokens', 'translation', 'size', 'rotation', 'prev', 'next', 'num_lidar_pts', 'num_radar_pts', 'category_name'])
[358.212, 1081.729, 0.756]
[0.8342998112464631, 0.0, 0.0, 0.5513110056529944]


In [83]:
from EKF.generate_GT import *
corrected_data = []
GT_data        = []

for index in tqdm(range(len(training_data))):
    #print("Current track ", index)
    EKF_GT_obj =  EKF_GT()
    corrected_data.append(EKF_GT_obj.correct_heading(training_data[index]))
    EKF_GT_obj.run_ekf()
    GT_data.append(EKF_GT_obj.KalmanFilter.afterSmooth.x)
    #EKF_GT_obj.plot_result()
    

100%|██████████| 934/934 [00:29<00:00, 31.34it/s]


## Save the GT data

In [84]:
correcte_measurement_file_name = "/media/liangxu/ArmyData/nuscenes/Tracking_result/tracking_tmp/training_data_moving_corrected"
GT_data_file_name = "/media/liangxu/ArmyData/nuscenes/Tracking_result/tracking_tmp/training_data_moving_GT"

In [85]:
with open(correcte_measurement_file_name, 'wb') as f:
    pickle.dump(corrected_data, f)
with open(GT_data_file_name, 'wb') as f:
    pickle.dump(GT_data, f)